In [ ]:
import importlib
import math
import matplotlib.pyplot as plt
import numpy as np
import os
from obspy import read
from obspy import UTCDateTime
from scipy import stats
from scipy.fft import rfft, rfftfreq, fft
import statistics
import utils
importlib.reload(utils)
import copy
import re
import random
from itertools import combinations

In [ ]:
def frequency_list(): 
    file = 'CC_for_ZZ_forest/REF/ZZ/9M_8_9M_9.MSEED'
    data = read(file)[0]
    freqs = rfftfreq(len(data), 1/data.meta['sampling_rate'])
    mask = (freqs > 10) & (freqs < 100)
    filtered_freqs = freqs[mask]
    return filtered_freqs

def station_list(folder):
    all_cc_pairs = [item for item in os.listdir(folder) if not item.startswith('.')]
    unique_numbers = list({int(num) for item in all_cc_pairs for num in re.findall(r'9M_(\d+)', item)})
    return unique_numbers

def normalize_complex_vector(vector):
    vector = np.asarray(vector, dtype=complex)
    magnitude = np.linalg.norm(vector)
    if magnitude == 0:
        raise ValueError("Cannot normalize a zero vector.")
    return vector / magnitude

def product(v1, v2):
    v1 = np.asarray(v1, dtype=complex)
    v2 = np.asarray(v2, dtype=complex)
    s1 = (v1 * v1.conj()).real
    s2 = (v2 * v2.conj()).real
    num = np.einsum('i,i->', np.sqrt(s1), np.sqrt(s2))
    den = np.sqrt(np.einsum('i->', s1)) * np.sqrt(np.einsum('i->', s2))
    c = np.clip(num / den, -1.0, 1.0).real
    return float(np.arccos(c))

def do_fft(folder, ref_sta_index):
    all_cc_pairs = [item for item in os.listdir(folder) if not item.startswith('.')]
    related_pairs = [
        pair for pair in all_cc_pairs
        if ref_sta_index in map(int, re.findall(r'9M_(\d+)', pair))
    ]

    all_amp = []
    for pair in related_pairs:
        file_path = folder + '/' + pair
        data = read(file_path)[0]
        freqs = rfftfreq(len(data.data), data.stats.delta)
        all_amp.append(rfft(data.data)[(10 <= freqs) & (freqs <= 100)].tolist())

    reorder_fft_results = dict()
    for i, freq in enumerate(frequency_list()):
        reorder_fft_results[freq] = dict()
        for j, pair in enumerate(related_pairs):
            reorder_fft_results[freq][pair] = all_amp[j][i]
    
    return reorder_fft_results

def get_phase_change(folder, ref_sta_index):
    fft_results = do_fft(folder, ref_sta_index)
    all_cc_pairs = list(next(iter(fft_results.values())).keys())
    unique_combination = list(combinations(all_cc_pairs, vector_size))
    random_selected_combination = random.sample(unique_combination, 400)

    state_vector = dict()
    for freq in frequency_list():
        state_vector[freq] = dict()
        for combination in random_selected_combination:
            state_vector[freq][combination] = []
            for pair in combination:
                state_vector[freq][combination].append(fft_results[freq][pair])
            state_vector[freq][combination] = normalize_complex_vector(state_vector[freq][combination])

    reference_state_vector = {
    f: np.vstack([((np.asarray(v, complex) * np.asarray(v, complex).conjugate()).real)
                    for v in sub.values()]).mean(axis=0)
    for f, sub in state_vector.items() if sub}

    phase_change = copy.deepcopy(state_vector)
    for freq in phase_change:
        for combination in phase_change[freq]:
            yita = product(state_vector[freq][combination], reference_state_vector[freq])
            phase_change[freq][combination] = yita

    return state_vector, phase_change

In [ ]:
tree = "CC_for_ZZ_forest/REF/ZZ"
nontree = "CC_for_ZZ_nonforest/REF/ZZ"

tree_references = station_list(tree)
# tree_references = [258, 131, 260, 261, 134, 263, 8, 9, 12, 17, 148, 29, 32, 39, 42, 44, 47, 55, 60, 61, 62, 72, 74, 76, 84, 90, 247, 102, 109, 243, 119, 125, 254]
nontree_references = station_list(nontree)
# nontree_references = [128, 129, 130, 141, 142, 143, 144, 145, 38, 63, 64, 65, 66, 67, 68, 79, 80, 81, 82, 83, 96, 97, 98, 99, 100, 101, 111, 112, 113, 114, 115, 126, 127]

all_mean_tree = []
all_mean_nontree = []

whole_freq = list(frequency_list())

vector_size = 4

for ref in tree_references:
    print(f'working on {ref}')
    state_vector_tree, phase_change_tree = get_phase_change(tree, ref)
    mean_tree = []
    for freq in whole_freq:
        mean_tree.append(np.mean(list(phase_change_tree[freq].values())))
    all_mean_tree.append(mean_tree)

all_mean_tree = [list(row) for row in zip(*all_mean_tree)]

tree_yita = []
tree_yita_var = []
for sublist in all_mean_tree:
    tree_yita.append(np.mean(sublist))
    tree_yita_var.append(np.var(sublist, ddof=1))

for ref in nontree_references:
    print(f'working on {ref}')
    state_vector_nontree, phase_change_nontree = get_phase_change(nontree, ref)
    mean_nontree = []
    for freq in whole_freq:
        mean_nontree.append(np.mean(list(phase_change_nontree[freq].values())))
    all_mean_nontree.append(mean_nontree)

all_mean_nontree = [list(row) for row in zip(*all_mean_nontree)]

nontree_yita = []
nontree_yita_var = []
for sublist in all_mean_nontree:
    nontree_yita.append(np.mean(sublist))

plt.figure(figsize=(6.5, 6.5))
plt.plot(whole_freq, tree_yita, label = f'Forest', color = '#00b8ff')
plt.plot(whole_freq, nontree_yita, label = f'Non-forest', color = '#ff952c')
plt.ylim(0.35, 0.63)
plt.xlabel('Frequency (Hz)', fontsize = 16)
plt.ylabel('Average $\\Delta\\eta$ (rad)', fontsize = 16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)
plt.savefig("results/Average_phase_change_4.svg", format="svg", dpi=300, bbox_inches="tight")

In [ ]:
tree = "CC_for_ZZ_forest/REF/ZZ"
nontree = "CC_for_ZZ_nonforest/REF/ZZ"

tree_references = station_list(tree)
# tree_references = [258, 131, 260, 261, 134, 263, 8, 9, 12, 17, 148, 29, 32, 39, 42, 44, 47, 55, 60, 61, 62, 72, 74, 76, 84, 90, 247, 102, 109, 243, 119, 125, 254]
nontree_references = station_list(nontree)
# nontree_references = [128, 129, 130, 141, 142, 143, 144, 145, 38, 63, 64, 65, 66, 67, 68, 79, 80, 81, 82, 83, 96, 97, 98, 99, 100, 101, 111, 112, 113, 114, 115, 126, 127]

all_mean_tree = []
all_mean_nontree = []

whole_freq = list(frequency_list())

vector_size = 7

for ref in tree_references:
    print(f'working on {ref}')
    state_vector_tree, phase_change_tree = get_phase_change(tree, ref)
    mean_tree = []
    for freq in whole_freq:
        mean_tree.append(np.mean(list(phase_change_tree[freq].values())))
    all_mean_tree.append(mean_tree)

all_mean_tree = [list(row) for row in zip(*all_mean_tree)]

tree_yita = []
tree_yita_var = []
for sublist in all_mean_tree:
    tree_yita.append(np.mean(sublist))
    tree_yita_var.append(np.var(sublist, ddof=1))

for ref in nontree_references:
    print(f'working on {ref}')
    state_vector_nontree, phase_change_nontree = get_phase_change(nontree, ref)
    mean_nontree = []
    for freq in whole_freq:
        mean_nontree.append(np.mean(list(phase_change_nontree[freq].values())))
    all_mean_nontree.append(mean_nontree)

all_mean_nontree = [list(row) for row in zip(*all_mean_nontree)]

nontree_yita = []
nontree_yita_var = []
for sublist in all_mean_nontree:
    nontree_yita.append(np.mean(sublist))
    nontree_yita_var.append(np.var(sublist, ddof=1))

plt.figure(figsize=(6.5, 6.5))
plt.plot(whole_freq, tree_yita, label = f'Forest', color = '#00b8ff')
plt.plot(whole_freq, nontree_yita, label = f'Non-forest', color = '#ff952c')
plt.ylim(0.35, 0.63)
plt.xlabel('Frequency (Hz)', fontsize = 16)
plt.ylabel('Average $\\Delta\\eta$ (rad)', fontsize = 16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)
plt.savefig("results/Average_phase_change_7.svg", format="svg", dpi=300, bbox_inches="tight")

In [ ]:
tree = "CC_for_ZZ_forest/REF/ZZ"
nontree = "CC_for_ZZ_nonforest/REF/ZZ"

tree_references = station_list(tree)
# tree_references = [258, 131, 260, 261, 134, 263, 8, 9, 12, 17, 148, 29, 32, 39, 42, 44, 47, 55, 60, 61, 62, 72, 74, 76, 84, 90, 247, 102, 109, 243, 119, 125, 254]
nontree_references = station_list(nontree)
# nontree_references = [128, 129, 130, 141, 142, 143, 144, 145, 38, 63, 64, 65, 66, 67, 68, 79, 80, 81, 82, 83, 96, 97, 98, 99, 100, 101, 111, 112, 113, 114, 115, 126, 127]

all_mean_tree = []
all_mean_nontree = []

whole_freq = list(frequency_list())

vector_size = 10

for ref in tree_references:
    print(f'working on {ref}')
    state_vector_tree, phase_change_tree = get_phase_change(tree, ref)
    mean_tree = []
    for freq in whole_freq:
        mean_tree.append(np.mean(list(phase_change_tree[freq].values())))
    all_mean_tree.append(mean_tree)

all_mean_tree = [list(row) for row in zip(*all_mean_tree)]

tree_yita = []
tree_yita_var = []
for sublist in all_mean_tree:
    tree_yita.append(np.mean(sublist))
    tree_yita_var.append(np.var(sublist, ddof=1))

for ref in nontree_references:
    print(f'working on {ref}')
    state_vector_nontree, phase_change_nontree = get_phase_change(nontree, ref)
    mean_nontree = []
    for freq in whole_freq:
        mean_nontree.append(np.mean(list(phase_change_nontree[freq].values())))
    all_mean_nontree.append(mean_nontree)

all_mean_nontree = [list(row) for row in zip(*all_mean_nontree)]

nontree_yita = []
nontree_yita_var = []
for sublist in all_mean_nontree:
    nontree_yita.append(np.mean(sublist))
    nontree_yita_var.append(np.var(sublist, ddof=1))

plt.figure(figsize=(6.5, 6.5))
plt.plot(whole_freq, tree_yita, label = f'Forest', color = '#00b8ff')
plt.plot(whole_freq, nontree_yita, label = f'Non-forest', color = '#ff952c')
plt.ylim(0.35, 0.63)
plt.xlabel('Frequency (Hz)', fontsize = 16)
plt.ylabel('Average $\\Delta\\eta$ (rad)', fontsize = 16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=14)
plt.savefig("results/Average_phase_change_10.svg", format="svg", dpi=300, bbox_inches="tight")

## Convergence test: The proper number of combinationination to choose

In [ ]:
vector_size = 10
tree = "CC_for_ZZ_forest/REF/ZZ"
whole_freq = list(frequency_list())
picked_freq = [whole_freq[i] for i in np.linspace(0, len(whole_freq) - 1, 540, dtype=int)]
nums = [5, 10, 25, 50, 75, 100, 150, 200, 250, 300]

total_var = []
for num in nums:
    print(f'working on', num)
    def get_phase_change(folder, ref_sta_index):
        fft_results = do_fft(folder, ref_sta_index)
        all_cc_pairs = list(next(iter(fft_results.values())).keys())
        unique_combination = list(combinations(all_cc_pairs, vector_size))
        random_selected_combination = random.sample(unique_combination, num)

        state_vector = dict()
        for freq in frequency_list():
            state_vector[freq] = dict()
            for combination in random_selected_combination:
                state_vector[freq][combination] = []
                for pair in combination:
                    state_vector[freq][combination].append(fft_results[freq][pair])
                state_vector[freq][combination] = normalize_complex_vector(state_vector[freq][combination])

        reference_state_vector = {
        f: np.vstack([((np.asarray(v, complex) * np.asarray(v, complex).conjugate()).real)
                        for v in sub.values()]).mean(axis=0)
        for f, sub in state_vector.items() if sub}

        phase_change = copy.deepcopy(state_vector)
        for freq in phase_change:
            for combination in phase_change[freq]:
                yita = product(state_vector[freq][combination], reference_state_vector[freq])
                phase_change[freq][combination] = yita

        return state_vector, phase_change

    tree_references = [258]
    #[84,32,243,39,72,260,258,62]
    total_mean_tree = []

    for i in range(10):
        state_vector_tree, phase_change_tree = get_phase_change(tree, 258)
        mean_tree = []

        for freq in whole_freq:
            tree_yita = []
            for combination in phase_change_tree[freq]:
                tree_yita.append(phase_change_tree[freq][combination])
            mean_tree_yita = np.mean(tree_yita)
            mean_tree.append(mean_tree_yita)
        total_mean_tree.append(mean_tree)

    transposed_total_mean_tree = [list(row) for row in zip(*total_mean_tree)]

    v = []

    for row in transposed_total_mean_tree:
        variance = np.var(row, ddof=1)
        v.append(variance)
    total_var.append(v)

plt.figure(figsize=(6.5, 6.5))
transposed_v = [list(row) for row in zip(*total_var)]
for row in transposed_v:
    plt.plot(nums, row, color = '#00b8ff', alpha = 0.4, linewidth = 0.3)

plt.xlabel('Number of combinationination', fontsize = 18)
plt.ylabel('Variance of $\\Delta\\eta$', fontsize = 18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.savefig("results/Convergence_test_tree.svg", format="svg", dpi=300, bbox_inches="tight")


In [ ]:
vector_size = 10
nontree = "DPZ_CC_for_ZZ_nonforest/REF/ZZ"
whole_freq = list(frequency_list())
picked_freq = [whole_freq[i] for i in np.linspace(0, len(whole_freq) - 1, 540, dtype=int)]
nums = [5, 10, 25, 50, 75, 100, 150, 200, 250, 300]

total_var = []
for num in nums:
    print(f'working on', num)
    def get_phase_change(folder, ref_sta_index):
        fft_results = do_fft(folder, ref_sta_index)
        all_cc_pairs = list(next(iter(fft_results.values())).keys())
        unique_combination = list(combinations(all_cc_pairs, vector_size))
        random_selected_combination = random.sample(unique_combination, num)

        state_vector = dict()
        for freq in frequency_list():
            state_vector[freq] = dict()
            for combination in random_selected_combination:
                state_vector[freq][combination] = []
                for pair in combination:
                    state_vector[freq][combination].append(fft_results[freq][pair])
                state_vector[freq][combination] = normalize_complex_vector(state_vector[freq][combination])
        
        reference_state_vector = {
        f: np.vstack([((np.asarray(v, complex) * np.asarray(v, complex).conjugate()).real)
                        for v in sub.values()]).mean(axis=0)
        for f, sub in state_vector.items() if sub}

        phase_change = copy.deepcopy(state_vector)
        for freq in phase_change:
            for combination in phase_change[freq]:
                yita = product(state_vector[freq][combination], reference_state_vector[freq])
                phase_change[freq][combination] = yita

        return state_vector, phase_change

    nontree_references = [83]
    #[84,32,243,39,72,260,258,62]
    total_mean_tree = []

    for i in range(10):
        state_vector_tree, phase_change_tree = get_phase_change(nontree, 83)
        mean_tree = []

        for freq in whole_freq:
            tree_yita = []
            for combination in phase_change_tree[freq]:
                tree_yita.append(phase_change_tree[freq][combination])
            mean_tree_yita = np.mean(tree_yita)
            mean_tree.append(mean_tree_yita)
        total_mean_tree.append(mean_tree)

    transposed_total_mean_tree = [list(row) for row in zip(*total_mean_tree)]

    v = []

    for row in transposed_total_mean_tree:
        variance = np.var(row, ddof=1)
        v.append(variance)
    total_var.append(v)

plt.figure(figsize=(6.5, 6.5))
transposed_v = [list(row) for row in zip(*total_var)]
for row in transposed_v:
    plt.plot(nums, row, color = '#ff952c', alpha = 0.4, linewidth = 0.3)

plt.xlabel('Number of combinationination', fontsize = 18)
plt.ylabel('Variance of $\\Delta\\eta$', fontsize = 18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.savefig("results/Convergence_test_nontree.svg", format="svg", dpi=300, bbox_inches="tight")